In [86]:
from PIL import Image
import numpy as np
import os

# ============================================================================
# DNA FONT RENDERING
# ============================================================================

BASE_IMAGES = {
    "A": {"upper": 'DNA_FONT/adenine_font_upper.png', "lower": 'DNA_FONT/adenine_font_lower.png'},
    "T": {"upper": 'DNA_FONT/thymine_font_upper.png', "lower": 'DNA_FONT/thymine_font_lower.png'},
    "C": {"upper": 'DNA_FONT/cytosine_font_upper.png', "lower": 'DNA_FONT/cytosine_font_lower.png'},
    "G": {"upper": 'DNA_FONT/guanine_font_upper.png', "lower": 'DNA_FONT/guanine_font_lower.png'},
}

HORIZONTAL_STEP = 160
VERTICAL_SHIFT = 220
OVERLAP_X_LOWER_PURINES = -38
OVERLAP_X_HIGHER_PURINES = -22
OVERLAP_X_LOWER_PYRIMIDINES = -60
OVERLAP_X_HIGHER_PYRIMIDINES = -22
ADDITIONAL_KICK_LOWER_PYRIMIDINES = 21
MARGIN = 50

# def render_dna_sequence(seq, output_file=None):
#     """Render a DNA sequence to an image. Returns the Image object."""
#     images = []
    
#     for i, base in enumerate(seq):
#         pos = "upper" if i % 2 == 0 else "lower"
#         img_path = BASE_IMAGES[base][pos]
#         img = Image.open(img_path).convert("RGBA")
#         images.append((img, pos, base))
    
#     width = HORIZONTAL_STEP * (len(seq) - 1) + images[0][0].width + MARGIN * 2
#     height = max(img[0].height for img in images) + VERTICAL_SHIFT + MARGIN * 2
#     canvas = Image.new("RGBA", (width, height), (255, 255, 255, 0))
    
#     prev_x = MARGIN
#     for i, (img, pos, base) in enumerate(images):
#         overlap = 0
#         additional_kick = 0
#         if base in ['A', 'G']:
#             if pos == "upper":
#                 overlap = OVERLAP_X_HIGHER_PURINES
#             else:
#                 overlap = OVERLAP_X_LOWER_PURINES
#         elif base in ['C', 'T']:
#             if pos == "upper":
#                 overlap = OVERLAP_X_HIGHER_PYRIMIDINES
#             else:
#                 overlap = OVERLAP_X_LOWER_PYRIMIDINES
#                 additional_kick = ADDITIONAL_KICK_LOWER_PYRIMIDINES
#         prev_x = prev_x + additional_kick
#         x = prev_x
#         prev_x = prev_x + HORIZONTAL_STEP + overlap
#         y = MARGIN if pos == "upper" else MARGIN + VERTICAL_SHIFT
#         canvas.alpha_composite(img, (x, y))
    
#     if output_file:
#         canvas.save(output_file)
#         print(f"Saved: {output_file}")
    
#     return canvas

def render_dna_sequence(seq, output_file=None, crop=True):
    """Render a DNA sequence to an image. Returns the Image object."""
    images = []
    
    for i, base in enumerate(seq):
        pos = "upper" if i % 2 == 0 else "lower"
        img_path = BASE_IMAGES[base][pos]
        img = Image.open(img_path).convert("RGBA")
        images.append((img, pos, base))
    
    width = HORIZONTAL_STEP * (len(seq) - 1) + images[0][0].width + MARGIN * 2
    height = max(img[0].height for img in images) + VERTICAL_SHIFT + MARGIN * 2
    canvas = Image.new("RGBA", (width, height), (255, 255, 255, 0))
    
    prev_x = MARGIN
    max_x = 0  # Track the rightmost pixel
    
    for i, (img, pos, base) in enumerate(images):
        overlap = 0
        additional_kick = 0
        if base in ['A', 'G']:
            if pos == "upper":
                overlap = OVERLAP_X_HIGHER_PURINES
            else:
                overlap = OVERLAP_X_LOWER_PURINES
        elif base in ['C', 'T']:
            if pos == "upper":
                overlap = OVERLAP_X_HIGHER_PYRIMIDINES
            else:
                overlap = OVERLAP_X_LOWER_PYRIMIDINES
                additional_kick = ADDITIONAL_KICK_LOWER_PYRIMIDINES
        prev_x = prev_x + additional_kick
        x = prev_x
        prev_x = prev_x + HORIZONTAL_STEP + overlap
        y = MARGIN if pos == "upper" else MARGIN + VERTICAL_SHIFT
        canvas.alpha_composite(img, (x, y))
        
        # Track rightmost extent
        max_x = max(max_x, x + img.width)
    
    # Crop to actual content with margin
    if crop:
        canvas = canvas.crop((0, 0, max_x + MARGIN, height))
    
    if output_file:
        canvas.save(output_file)
        print(f"Saved: {output_file}")
    
    return canvas



# ============================================================================
# RING BENDING
# ============================================================================

def bend_to_circle(img, inner_radius=200, outer_radius=600):
    """Warp a horizontal image into a ring shape using polar transform."""
    img = img.convert("RGBA")
    w, h = img.size
    
    out_r = outer_radius
    out_size = out_r * 2
    
    src = np.array(img)
    dst = np.zeros((out_size, out_size, 4), dtype=np.uint8)
    
    center = out_size // 2
    ring_thickness = outer_radius - inner_radius
    
    for y in range(out_size):
        for x in range(out_size):
            dx = x - center
            dy = y - center
            r = np.sqrt(dx*dx + dy*dy)
            if inner_radius <= r < outer_radius:
                theta = np.arctan2(dy, dx)
                if theta < 0:
                    theta += 2 * np.pi
                src_x = int(theta / (2 * np.pi) * w)
                src_y = int((r - inner_radius) / ring_thickness * h)
                if 0 <= src_x < w and 0 <= src_y < h:
                    dst[y, x] = src[src_y, src_x]
    
    return Image.fromarray(dst, "RGBA")

# ============================================================================
# MAIN COMBINED FUNCTION
# ============================================================================

def split_sequence_for_rings(sequence, innermost_radius=100, ring_height=50, margin=10):
    """
    Split a DNA sequence into chunks appropriate for each ring.
    Returns list of (start_idx, end_idx) tuples for the sequence.
    """
    # Calculate approximate width per nucleotide (considering overlaps)
    avg_width_per_base = HORIZONTAL_STEP - 50  # rough estimate with overlaps
    
    chunks = []
    current_inner = innermost_radius
    current_idx = 0
    
    while current_idx < len(sequence):
        inner_r = current_inner
        outer_r = inner_r + ring_height
        
        # Calculate ring area
        ring_area = np.pi * (outer_r**2 - inner_r**2)
        
        # Estimate height of rendered DNA strip
        estimated_height = VERTICAL_SHIFT + MARGIN * 2 + 200  # approximate
        
        # Width needed for this ring
        strip_width = int(ring_area / estimated_height)
        
        # How many bases fit?
        bases_for_ring = max(1, int(strip_width / avg_width_per_base))
        
        # Don't exceed sequence length
        end_idx = min(current_idx + bases_for_ring, len(sequence))
        
        chunks.append((current_idx, end_idx))
        
        current_idx = end_idx
        current_inner = outer_r + margin
        
        if current_idx >= len(sequence):
            break
    
    return chunks

def create_dna_concentric_rings(sequence, innermost_radius=100, ring_height=50, 
                                 margin=10, output_file="dna_rings.png",
                                 save_intermediates=False):
    """
    Create concentric rings from a DNA sequence.
    
    Parameters:
    - sequence: DNA sequence string (e.g., "ATCGATCG")
    - innermost_radius: radius of innermost ring's inner edge
    - ring_height: thickness of each ring
    - margin: spacing between rings
    - output_file: path to save final output
    - save_intermediates: if True, save intermediate linear strips
    
    Returns:
    - Final composite image
    """
    sequence = sequence.upper()
    print(f"Processing DNA sequence: {sequence}")
    print(f"Total length: {len(sequence)} nucleotides")
    
    # Split sequence into chunks for rings
    chunks = split_sequence_for_rings(sequence, innermost_radius, ring_height, margin)
    print(f"Split into {len(chunks)} rings")
    
    rings = []
    current_inner = innermost_radius
    
    for i, (start_idx, end_idx) in enumerate(chunks):
        if i == len(chunks) - 1:
            continue

        chunk_seq = sequence[start_idx:end_idx]
        print(f"Ring {i+1}: bases {start_idx}-{end_idx} ({len(chunk_seq)} bases)")
        
        # Render this chunk to a linear strip
        strip_img = render_dna_sequence(
            chunk_seq,
            output_file=f"ring_{i+1}_linear.png" if save_intermediates else None,
            crop= i != len(chunks) - 1 # Don't crop the last one
        )
        
        # Calculate ring dimensions
        inner_r = current_inner
        outer_r = inner_r + ring_height
        
        # Bend to circle
        ring = bend_to_circle(strip_img, inner_radius=inner_r, outer_radius=outer_r)
        rings.append(ring)
        
        if save_intermediates:
            ring.save(f"ring_{i+1}_circular.png")
        
        current_inner = outer_r + margin
    
    # Composite all rings
    if not rings:
        print("No rings created!")
        return None
    
    max_size = max(ring.size[0] for ring in rings)
    final_img = Image.new("RGBA", (max_size, max_size), (255, 255, 255, 0))
    
    for ring in rings:
        offset = (max_size - ring.size[0]) // 2
        final_img.paste(ring, (offset, offset), ring)
    
    final_img.save(output_file)
    print(f"Saved final result: {output_file}")
    
    return final_img


In [87]:
# TODO: have them all start with the shine diagora sequence...

# fourXYZ =  'AAGGAGGACGACAAATGGGACTGGCCCTTAAGTTAGGCATCAGTGCCGACGCCGCTTATAAACTTGCTAAAGCAGCCTACTATTCAGGAACTACCGTTGAAGAAGCCTACAAATTGGCTCTTAAGTTGGGAATATCCGCTGATGCGGCTTATAAGCTTGCGGAAGCAGCCTACTATAGTGGAACCACTGTGGAAGAGGCCTATAAACTTGCATTAAAACTGGGGATCTCCGCTGATGCTGCATACAAATTAGCAAAAGCGGCGTACTACTCGGGCACTACCGTTGAAGAAGCATATAAGTTAGCACTGAAACTCGGGATCAGCGCGGACGCGGCTTACAAACTTGCAAAGGCGGCCTACTACTCGGGAACGACAGTGGAAGAGGCATATAAGCTTGCACTTAAACTGGGCATTAGTGCCGACGCAGCGTACAAGTTAGCCAAAGCCGCATATTATTCAGGTACGACTGTTGAAGAGGCATACAAATTAGCACTGAAGCTCGGCATATCCGCGGACGCCGCGTATAAACTGGCTAAAGCTGCTTACTATAGCGGCACGACAGTAGAGGAAGCTTACAAATTGGCCCTCAAGTTAGGTATCTCGGCGGATGCCGCGTATAAACTTGCAAAAGCGGCGTATTACTCAGGCACAACCGTTGAGGAAGCTTATAAATTGGCATTGAAATTAGGCATAAGTGCCGACGCTGCTTACAAACTGGCCAAGGCCGCGTACTACTCGGGAACAACGGTCGAAGAGGCATACAAGCTGGCATTAAAGTTAGGCATTAGCGCGGATGCTGCTTATAAATTGGCCAAAGCAGCATACTATTCTGGGACAACTGTCGAGGAAGCGTACAAGCTGGCACTGAAGTTGGGCTGA'
# eightE2Q = 'AAAGAGGAGAAAACACATCATGTCTGAAGAGCGGGTGGAAGAAGAACTCAGACGTCTCGTTGAAGAAGAGGCCGAGGCAAGAGGTATCTCACGGCGTGAGACACTTGAGCGGTTATCATTGTTATTGTTCTCGATGCAACTTGAGAAGCTTGTCAAGGAAGAGGCAGAGGCTCGGGGAGTCTCGGTAGAGACTATCAGAGAGGAGCTTGAGCGTGAGGTAGACGAGCGATTAAGAGAGCTCCGGGAAGAGGGGATCTCACGACGTGAGACTCTTGAGCGTCTCTCTTTGCTCCTTTTCTCAATGCAATTAGAGAAGCTTGTAAAGGAAGAGGCCGAGGCCCGTGGAGTTTCTGTTGAGACAATAAGAGAGGAGTTAGAGAGAGAGGTAGACGAGCGATTGCGGGAGCTCCGTGAAGAGGGTATATCACGTCGAGAGACTCTCGAGCGTCTTTCCTTACTTTTATTCAGTATGCAACTTGAGAAGTTGGTCAAGGAAGAGGCAGAGGCCAGAGGGGTAAGTGTCGAGGAGATCCGTGAGGAGTTGCGTCGTGAGGTAGACGAGCGGCTCCGTGAGTTACGTGAGGAAGGTATCTCAAGAAGAGAGACGCTCGAGCGTCTCTCGCTTTTACTCTTCTCAATGCAATTAGAGAAGCTTGTCAAGGAAGAGGCTGAGGCTCGGGGTGTTTCTGTCGAGGAGATCCGTGAGGAGCTTAGACGTGAGGTAGACGAGCGGCTCCGTGAGCTTCGGGAAGAGGGTATATCAAGACGTGAGACTTTAGAGCGGTTGTCCCTTCTTTTATTCTCTATGCAATTAGAGAAGTTAGTTAAGGAAGAGGCCGAGGCTAGAGGTGTTAGTGTAGAGGAGATCCGTGAGGAGCTCCGGAGAGAGGTTGAGGAGCGTTTACGTGAGTTGCGTGAAGAGGGTGAGTGA'
eightE2Q_full_plasmid = 'GTTGCGGCCACCGGCTGGCTCGCTTCGCTCGGCCCGTGGACAACCCTGCTGGACAAGCTGATGGACAGGCTGCGCCTGCCCACGAGCTTGACCACAGGGATTGCCCACCGGCTACCCAGCCTTCGACCACATACCCACCGGCTCCAACTGCGCGGCCTGCGGCCTTGCCCCATCAATTTTTTTAATTTTCTCTGGGGAAAAGCCTCCGGCCTGCGGCCTGCGCGCTTCGCTTGCCGGTTGGACACCAAGTGGAAGGCGGGTCAAGGCTCGCGCAGCGACCGCGCAGCGGCTTGGCCTTGACGCGCCTGGAACGACCCAAGCCTATGCGAGTGGGGGCAGTCGAAGGCGAAGCCCGCCCGCCTGCCCCCCGAGCCTCACGGCGGCGAGTGCGGGGGTTCCAAGGGGGCAGCGCCACCTTGGGCAAGGCCGAAGGCCGCGCAGTCGATCAACAAGCCCCGGAGGGGCCACTTTTTGCCGGAGGGGGAGCCGCGCCGAAGGCGTGGGGGAACCCCGCAGGGGTGCCCTTCTTTGGGCACCAAAGAACTAGATATAGGGCGAAATGCGAAAGACTTAAAAATCAACAACTTAAAAAAGGGGGGTACGCAACAGCTCATTGCGGCACCCCCCGCAATAGCTCATTGCGTAGGTTAAAGAAAATCTGTAATTGACTGCCACTTTTACGCAACGCATAATTGTTGTCGCGCTGCCGAAAAGTTGCAGCTGATTGCGCATGGTGCCGCAACCGTGCGGCACCCTACCGCATGGAGATAAGCGATGATCGATCGATCGATCGATTCGATCGATCGATCGATCGCGAGAGGGCAGGATAGGTGAAGTAGGCCCACCCGCGAGCGGGTGTTCCTTCTTCACTGTCCCTTATTCGCACCTGGCGGTGCTCAACGGGAATCCTGCTCTGCGAGGCTGGCCGGGATCGATCGATCGATCGATTCGATCGATCGATCGATCGCGAGAttgacggctagctcagtcctaggtacagtgctagcATCGATCGATCGAAAGAGGAGAAAACACATCATGGCCACGCAGTCCAGAGAAATCGGCATTCAAGCCAAGAACAAGCCCGGTCACTGGGTGCAAACGGAACGCAAAGCGCATGAGGCGTGGGCCGGGCTTATTGCGAGGAAACCCACGGCGGCAATGCTGCTGCATCACCTCGTGGCGCAGATGGGCCACCAGAACGCCGTGGTGGTCAGCCAGAAGACACTTTCCAAGCTCATCGGACGTTCTTTGCGGACGGTCCAATACGCAGTCAAGGACTTGGTGGCCGAGCGCTGGATCTCCGTCGTGAAGCTCAACGGCCCCGGCACCGTGTCGGCCTACGTGGTCAATGACCGCGTGGCGTGGGGCCAGCCCCGCGACCAGTTGCGCCTGTCGGTGTTCAGTGCCGCCGTGGTGGTTGATCACGACGACCAGGACGAATCGCTGTTGGGGCATGGCGACCTGCGCCGCATCCCGACCCTGTATCCGGGCGAGCAGCAACTACCGACCGGCCCCGGCGAGGAGCCGCCCAGCCAGCCCGGCATTCCGGGCATGGAACCAGACCTGCCAGCCTTGACCGAAACGGAGGAATGGGAACGGCGCGGGCAGCAGCGCCTGCCGATGCCCGATGAGCCGTGTTTTCTGGACGATGGCGAGCCGTTGGAGCCGCCGACACGGGTCACGCTGCCGCGCCGGTAGTGAGCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTCGCTAGCTAGCAGAAGGCCATCCTGACGGATGGCCTTTTATTGACGATCGATCGAAGTCGATTCGATCGATCGATCGATCGCGAGAttgacggctagctcagtcctaggtacagtgctagcATCGATCGATCGAAAGAGGAGAAAACACATCATGTCTGAAGAGCGGGTGGAAGAAGAACTCAGACGTCTCGTTGAAGAAGAGGCCGAGGCAAGAGGTATCTCACGGCGTGAGACACTTGAGCGGTTATCATTGTTATTGTTCTCGATGCAACTTGAGAAGCTTGTCAAGGAAGAGGCAGAGGCTCGGGGAGTCTCGGTAGAGACTATCAGAGAGGAGCTTGAGCGTGAGGTAGACGAGCGATTAAGAGAGCTCCGGGAAGAGGGGATCTCACGACGTGAGACTCTTGAGCGTCTCTCTTTGCTCCTTTTCTCAATGCAATTAGAGAAGCTTGTAAAGGAAGAGGCCGAGGCCCGTGGAGTTTCTGTTGAGACAATAAGAGAGGAGTTAGAGAGAGAGGTAGACGAGCGATTGCGGGAGCTCCGTGAAGAGGGTATATCACGTCGAGAGACTCTCGAGCGTCTTTCCTTACTTTTATTCAGTATGCAACTTGAGAAGTTGGTCAAGGAAGAGGCAGAGGCCAGAGGGGTAAGTGTCGAGGAGATCCGTGAGGAGTTGCGTCGTGAGGTAGACGAGCGGCTCCGTGAGTTACGTGAGGAAGGTATCTCAAGAAGAGAGACGCTCGAGCGTCTCTCGCTTTTACTCTTCTCAATGCAATTAGAGAAGCTTGTCAAGGAAGAGGCTGAGGCTCGGGGTGTTTCTGTCGAGGAGATCCGTGAGGAGCTTAGACGTGAGGTAGACGAGCGGCTCCGTGAGCTTCGGGAAGAGGGTATATCAAGACGTGAGACTTTAGAGCGGTTGTCCCTTCTTTTATTCTCTATGCAATTAGAGAAGTTAGTTAAGGAAGAGGCCGAGGCTAGAGGTGTTAGTGTAGAGGAGATCCGTGAGGAGCTCCGGAGAGAGGTTGAGGAGCGTTTACGTGAGTTGCGTGAAGAGGGTGAGTGAGCTAGCTAGCCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTCGCTAGCTAGCAGAAGGCCATCCTGACGGATGGCCTTTT'
# sevenSQ3 = 'AAAGAGGAGAAAATACATCATGGGCAGCAGCATGGGTTCAGACGAGCAACGACGTGAGCTTGAGGAGAAGATAAAGTTCAAGCTCGCTGAGTTGGCTTCAAAGTCAGAGGAAGAGCGGAAGGAGATCAAGCTCAGAGTTATAGCCTACGTTTTAGTTCAATTAGAGGACTTACAAAAGTTGTTATCTGACGAGCAACGTCGGGAGTTAGAGGAGAAGATAAAGTTCAAGCTTGCTGAGCTCGCCTCAAAGTCTGAGGAAGAGCGTAAGGAAATCAAGCTCAGAGTAATAGCATACGTTTTAGTTCAACTTGAGGACTTACAAAAGCTTTTATCTGACGAGCAACGTCGTGAGTTGGAAGAGAAGATAAAGTTCAAGTTAGCAGAGCTTGCTAGTAAGTCAGAAGAGGAGAGAAAGGAGATCAAGTTACGGGTTATAGCATACGTCTTGGTCCAACTTGAGGACTTACAAAAGTTACTCTCCTGA'
sevenSQ3_full_plasmid = 'AATGCTGCCATTTTTGGGGTGAGGCCGTTCGCGGCCGAGGGGCGCAGCCCCTGGGGGGATGGGAGGCCCGCGTTAGCGGGCCGGGAGGGTTCGAGAAGGGGGGGCACCCCCCTTCGGCGTGCGCGGTCACGCGCCAGGGCGCAGCCCTGGTTAAAAACAAGGTTTATAAATATTGGTTTAAAAGCAGGTTAAAAGACAGGTTAGCGGTGGCCGAAAAACGGGCGGAAACCCTTGCAAATGCTGGATTTTCTGCCTGTGGACAGCCCCTCAAATGTCAATAGGTGCGCCCCTCATCTGTCATCACTCTGCCCCTCAAGTGTCAAGGATCGCGCCCCTCATCTGTCAGTAGTCGCGCCCCTCAAGTGTCAATACCGCAGGGCACTTATCCCCAGGCTTGTCCACATCATCTGTGGGAAACTCGCGTAAAATCAGGCGTTTTCGCCGATTTGCGAGGCTGGCCAGCTCCACGTCGCCGGCCGAAATCGAGCCTGCCCCTCATCTGTCAACGCCGCGCCGGGTGAGTCGGCCCCTCAAGTGTCAACGTCCGCCCCTCATCTGTCAGTGAGGGCCAAGTTTTCCGCGTGGTATCCACAACGCCGGCGGCCGCGGTGTCTCGCACACGGCTTCGACGGCGTTTCTGGCGCGTTTGCAGGGCCATAGACGGCCGCCAGCCCAGCGGCGAGGGCAACCAGCCCGGTGAGCGTCGGAAAGGCGCGCTAGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAGGGCAGGATAGGTGAAGTAGGCCCACCCGCGAGCGGGTGTTCCTTCTTCACTGTCCCTTATTCGCACCTGGCGGTGCTCAACGGGAATCCTGCTCTGCGAGGCTGGCCGGTCACTCTAGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAAGCTTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGCAGTCGCGAAAGAGGAGAAAATACATCATGGGCAGCAGCATGGGTTCAGACGAGCAACGACGTGAGCTTGAGGAGAAGATAAAGTTCAAGCTCGCTGAGTTGGCTTCAAAGTCAGAGGAAGAGCGGAAGGAGATCAAGCTCAGAGTTATAGCCTACGTTTTAGTTCAATTAGAGGACTTACAAAAGTTGTTATCTGACGAGCAACGTCGGGAGTTAGAGGAGAAGATAAAGTTCAAGCTTGCTGAGCTCGCCTCAAAGTCTGAGGAAGAGCGTAAGGAAATCAAGCTCAGAGTAATAGCATACGTTTTAGTTCAACTTGAGGACTTACAAAAGCTTTTATCTGACGAGCAACGTCGTGAGTTGGAAGAGAAGATAAAGTTCAAGTTAGCAGAGCTTGCTAGTAAGTCAGAAGAGGAGAGAAAGGAGATCAAGTTACGGGTTATAGCATACGTCTTGGTCCAACTTGAGGACTTACAAAAGTTACTCTCCTGATCGATCTCACTAGCACAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTCGGCTCACCTTCGGGTGGGCCTTTCTGCGTTTATACCAGGCACTAGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCACTAGCTAGCGATCGATCGATCGATCGATCGATCGAAAGCTTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGCATCGATCGATTTCGATCGAAAGAGGAGAAAACTAATCATGAATCGGACGTTTGACCGGAAGGCATACAGGCAAGAACTGATCGACGCGGGGTTTTCCGCCGAGGATGCCGAAACCATCGCAAGCCGCACCGTCATGCGTGCGCCCCGCGAAACCTTCCAGTCCGTCGGCTCGATGGTCCAGCAAGCTACGGCCAAGATCGAGCGCGACAGCGTGCAACTGGCTCCCCCTGCCCTGCCCGCGCCATCGGCCGCCGTGGAGCGTTCGCGTCGTCTCGAACAGGAGGCGGCAGGTTTGGCGAAGTCGATGACCATCGACACGCGAGGAACTATGACGACCAAGAAGCGAAAAACCGCCGGCGAGGACCTGGCAAAACAGGTCAGCGAGGCCAAGCAGGCCGCGTTGCTGAAACACACGAAGCAGCAGATCAAGGAAATGCAGCTTTCCTTGTTCGATATTGCGCCGTGGCCGGACACGATGCGAGCGATGCCAAACGACACGGCCCGCTCTGCCCTGTTCACCACGCGCAACAAGAAAATCCCGCGCGAGGCGCTGCAAAACAAGGTCATTTTCCACGTCAACAAGGACGTGAAGATCACCTACACCGGCGTCGAGCTGCGGGCCGACGATGACGAACTGGTGTGGCAGCAGGTGTTGGAGTACGCGAAGCGCACCCCTATCGGCGAGCCGATCACCTTCACGTTCTACGAGCTTTGCCAGGACCTGGGCTGGTCGATCAATGGCCGGTATTACACGAAGGCCGAGGAATGCCTGTCGCGCCTACAGGCGACGGCGATGGGCTTCACGTCCGACCGCGTTGGGCACCTGGAATCGGTGTCGCTGCTGCACCGCTTCCGCGTCCTGGACCGTGGCAAGAAAACGTCCCGTTGCCAGGTCCTGATCGACGAGGAAATCGTCGTGCTGTTTGCTGGCGACCACTACACGAAATTCATATGGGAGAAGTACCGCAAGCTGTCGCCGACGGCCCGACGGATGTTCGACTATTTCAGCTCGCACCGGGAGCCGTACCCGCTCAAGCTGGAAACCTTCCGCCTCATGTGCGGATCGGATTCCACCCGCGTGAAGAAGTGGCGCGAGCAGGTCGGCGAAGCCTGCGAAGAGTTGCGAGGCAGCGGCCTGGTGGAACACGCCTGGGTCAATGATGACCTGGTGCATTGCAAACGCTAGTCGATCGATCACAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTCTACTAGAGTCACACTGGCTCACCTTCGGGTGGGCCTTTCTGCGTTTATA'
# fourXYZ = 'AATTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTCCCTTCGGGAAGCGTGGCGCTTTCTCATAGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGCTCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTATCCGGTAACTATCGTCTTGAGTCCAACCCGGTAAGACACGACTTATCGCCACTGGCAGCAGCCACTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGAGTTCTTGAAGTGGTGGCCTAACTACGGCTACACTAGAAGAACAGTATTTGGTATCTGCGCTCTGCTGAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATCCGGCAAACAAACCACCGCTGGTAGCGGTGGTTTTTTTGTTTGCAAGCAGCAGATTACGCGCAGAAAAAAAGGATCTCAAGCTAGCGCTAGCGCTAGTTCGAGCTCGGTACCCGGGGATCCCGCTAGCGCTAGCGCTAGCAGCTTCGTGCATGCAGGCCGGGGCATATGGGAAACAGCGCGGACGCAGCGGAATTTCCAATTTCATGCCGCAGCCGCCTGCGCTGTTCTCATTTGCGGCTTCCTTGTAGAGCTCAGCATTATTGAGTGGATGATTATATTCCTTTTGATAGGTGGTATGTTTTCGCTTGAACTTTTAAATACAGCCATTGAACATACGGTTGATTTAATAACTGACAAACATCACCCTCTTGCTAAAGCGGCCAAGGACGCTGCCGCCGGGGCTGTTTGCGTTTTTGCCGTGATTTCGTGTATCATTGGTTTACTTATTTTTTTGCCAAAGCTGTAATGGCTGAAAATTCTTACATTTATTTTACATTTTTAGAAATGGGCGTGAAAAAAAGCGCGCGATTATGTAAAATATAAAGTGATAGCGGTACCATTATAGGTAAGAGAGGAAAGTACACAATTCGAGCTCGGTACCCGGGGATCCAGGAGGACGACAAATGGGACTGGCCCTTAAGTTAGGCATCAGTGCCGACGCCGCTTATAAACTTGCTAAAGCAGCCTACTATTCAGGAACTACCGTTGAAGAAGCCTACAAATTGGCTCTTAAGTTGGGAATATCCGCTGATGCGGCTTATAAGCTTGCGGAAGCAGCCTACTATAGTGGAACCACTGTGGAAGAGGCCTATAAACTTGCATTAAAACTGGGGATCTCCGCTGATGCTGCATACAAATTAGCAAAAGCGGCGTACTACTCGGGCACTACCGTTGAAGAAGCATATAAGTTAGCACTGAAACTCGGGATCAGCGCGGACGCGGCTTACAAACTTGCAAAGGCGGCCTACTACTCGGGAACGACAGTGGAAGAGGCATATAAGCTTGCACTTAAACTGGGCATTAGTGCCGACGCAGCGTACAAGTTAGCCAAAGCCGCATATTATTCAGGTACGACTGTTGAAGAGGCATACAAATTAGCACTGAAGCTCGGCATATCCGCGGACGCCGCGTATAAACTGGCTAAAGCTGCTTACTATAGCGGCACGACAGTAGAGGAAGCTTACAAATTGGCCCTCAAGTTAGGTATCTCGGCGGATGCCGCGTATAAACTTGCAAAAGCGGCGTATTACTCAGGCACAACCGTTGAGGAAGCTTATAAATTGGCATTGAAATTAGGCATAAGTGCCGACGCTGCTTACAAACTGGCCAAGGCCGCGTACTACTCGGGAACAACGGTCGAAGAGGCATACAAGCTGGCATTAAAGTTAGGCATTAGCGCGGATGCTGCTTATAAATTGGCCAAAGCAGCATACTATTCTGGGACAACTGTCGAGGAAGCGTACAAGCTGGCACTGAAGTTGGGCTGAGCTAGCTAGCCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTCGCTAGCTAGCAGAAGGCCATCCTGACGGATGGCCTTTT'
fourXYZ_full_plasmid = 'AATTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTCCCTTCGGGAAGCGTGGCGCTTTCTCATAGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGCTCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTATCCGGTAACTATCGTCTTGAGTCCAACCCGGTAAGACACGACTTATCGCCACTGGCAGCAGCCACTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGAGTTCTTGAAGTGGTGGCCTAACTACGGCTACACTAGAAGAACAGTATTTGGTATCTGCGCTCTGCTGAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATCCGGCAAACAAACCACCGCTGGTAGCGGTGGTTTTTTTGTTTGCAAGCAGCAGATTACGCGCAGAAAAAAAGGATCTCAAGCTAGCGCTAGCGCTAGTTCGAGCTCGGTACCCGGGGATCCCGCTAGCGCTAGCGCTAGCAGCTTCGTGCATGCAGGCCGGGGCATATGGGAAACAGCGCGGACGCAGCGGAATTTCCAATTTCATGCCGCAGCCGCCTGCGCTGTTCTCATTTGCGGCTTCCTTGTAGAGCTCAGCATTATTGAGTGGATGATTATATTCCTTTTGATAGGTGGTATGTTTTCGCTTGAACTTTTAAATACAGCCATTGAACATACGGTTGATTTAATAACTGACAAACATCACCCTCTTGCTAAAGCGGCCAAGGACGCTGCCGCCGGGGCTGTTTGCGTTTTTGCCGTGATTTCGTGTATCATTGGTTTACTTATTTTTTTGCCAAAGCTGTAATGGCTGAAAATTCTTACATTTATTTTACATTTTTAGAAATGGGCGTGAAAAAAAGCGCGCGATTATGTAAAATATAAAGTGATAGCGGTACCATTATAGGTAAGAGAGGAAAGTACACAATTCGAGCTCGGTACCCGGGGATCCAGGAGGACGACAAATGGGACTGGCCCTTAAGTTAGGCATCAGTGCCGACGCCGCTTATAAACTTGCTAAAGCAGCCTACTATTCAGGAACTACCGTTGAAGAAGCCTACAAATTGGCTCTTAAGTTGGGAATATCCGCTGATGCGGCTTATAAGCTTGCGGAAGCAGCCTACTATAGTGGAACCACTGTGGAAGAGGCCTATAAACTTGCATTAAAACTGGGGATCTCCGCTGATGCTGCATACAAATTAGCAAAAGCGGCGTACTACTCGGGCACTACCGTTGAAGAAGCATATAAGTTAGCACTGAAACTCGGGATCAGCGCGGACGCGGCTTACAAACTTGCAAAGGCGGCCTACTACTCGGGAACGACAGTGGAAGAGGCATATAAGCTTGCACTTAAACTGGGCATTAGTGCCGACGCAGCGTACAAGTTAGCCAAAGCCGCATATTATTCAGGTACGACTGTTGAAGAGGCATACAAATTAGCACTGAAGCTCGGCATATCCGCGGACGCCGCGTATAAACTGGCTAAAGCTGCTTACTATAGCGGCACGACAGTAGAGGAAGCTTACAAATTGGCCCTCAAGTTAGGTATCTCGGCGGATGCCGCGTATAAACTTGCAAAAGCGGCGTATTACTCAGGCACAACCGTTGAGGAAGCTTATAAATTGGCATTGAAATTAGGCATAAGTGCCGACGCTGCTTACAAACTGGCCAAGGCCGCGTACTACTCGGGAACAACGGTCGAAGAGGCATACAAGCTGGCATTAAAGTTAGGCATTAGCGCGGATGCTGCTTATAAATTGGCCAAAGCAGCATACTATTCTGGGACAACTGTCGAGGAAGCGTACAAGCTGGCACTGAAGTTGGGCTGAGCTAGCTAGCCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTCGCTAGCTAGCAGAAGGCCATCCTGACGGATGGCCTTTT'

In [88]:
result = create_dna_concentric_rings(
    sequence=fourXYZ_full_plasmid.upper() + fourXYZ_full_plasmid.upper() + fourXYZ_full_plasmid.upper() + fourXYZ_full_plasmid.upper(),
    innermost_radius=int(900 * 1.5),
    ring_height=460,
    margin=10,
    output_file="10.8_inch_7SQ3_concentric_rings_finalhopefully.png",
    save_intermediates=True  
)

# 1000 == 12 inches

if result:
    print("Success! Created concentric DNA rings.")
else:
    print("Failed to create rings.")

Processing DNA sequence: AATTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTCCCTTCGGGAAGCGTGGCGCTTTCTCATAGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGCTCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTATCCGGTAACTATCGTCTTGAGTCCAACCCGGTAAGACACGACTTATCGCCACTGGCAGCAGCCACTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGAGTTCTTGAAGTGGTGGCCTAACTACGGCTACACTAGAAGAACAGTATTTGGTATCTGCGCTCTGCTGAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATCCGGCAAACAAACCACCGCTGGTAGCGGTGGTTTTTTTGTTTGCAAGCAGCAGATTACGCGCAGAAAAAAAGGATCTCAAGCTAGCGCTAGCGCTAGTTCGAGCTCGGTACCCGGGGATCCCGCTAGCGCTAGCGCTAGCAGCTTCGTGCATGCAGGCCGGGGCATATGGGAAACAGCGCGGACGCAGCGGAATTTCCAATTTCATGCCGCAGCCGCCTGCGCTGTTCTCATTTGCGGCTTCCTTGTAGAGCTCAGCATTATTGAGTGGATGATTATATTCCTTTTGATAGGTGGTATGTTTTCGCTTGAACTTTTAAATACAGCCATTGAACATACGGTTGATTTAATAACTGACAAACATCACCCTCTTGCTAAAGCGGCCAAGGACGCTGCCGCCGGGGCTGTTTGCGTTTTTGCCGTGATTTCGTGTATCATTGGTTTACTTATTTTTTTGCCAAAGCTG

/opt/miniconda3/envs/radial_faces_env/lib/python3.8/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (91223617 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Saved: ring_24_linear.png
Saved final result: 10.8_inch_7SQ3_concentric_rings_finalhopefully.png
Success! Created concentric DNA rings.


In [89]:
# Let's triple the DNA, so that way we can get it to go all the way down the linenn...
result = create_dna_concentric_rings(
    sequence=sevenSQ3_full_plasmid.upper() + sevenSQ3_full_plasmid.upper() + sevenSQ3_full_plasmid.upper() + sevenSQ3_full_plasmid.upper(),
    innermost_radius=int(750 * 1.5),
    ring_height=460,
    margin=10,
    output_file="9_inch_7SQ3_concentric_rings_finalhopefully.png",
    save_intermediates=True  
)

# 1000 == 12 inches

if result:
    print("Success! Created concentric DNA rings.")
else:
    print("Failed to create rings.")

Processing DNA sequence: AATGCTGCCATTTTTGGGGTGAGGCCGTTCGCGGCCGAGGGGCGCAGCCCCTGGGGGGATGGGAGGCCCGCGTTAGCGGGCCGGGAGGGTTCGAGAAGGGGGGGCACCCCCCTTCGGCGTGCGCGGTCACGCGCCAGGGCGCAGCCCTGGTTAAAAACAAGGTTTATAAATATTGGTTTAAAAGCAGGTTAAAAGACAGGTTAGCGGTGGCCGAAAAACGGGCGGAAACCCTTGCAAATGCTGGATTTTCTGCCTGTGGACAGCCCCTCAAATGTCAATAGGTGCGCCCCTCATCTGTCATCACTCTGCCCCTCAAGTGTCAAGGATCGCGCCCCTCATCTGTCAGTAGTCGCGCCCCTCAAGTGTCAATACCGCAGGGCACTTATCCCCAGGCTTGTCCACATCATCTGTGGGAAACTCGCGTAAAATCAGGCGTTTTCGCCGATTTGCGAGGCTGGCCAGCTCCACGTCGCCGGCCGAAATCGAGCCTGCCCCTCATCTGTCAACGCCGCGCCGGGTGAGTCGGCCCCTCAAGTGTCAACGTCCGCCCCTCATCTGTCAGTGAGGGCCAAGTTTTCCGCGTGGTATCCACAACGCCGGCGGCCGCGGTGTCTCGCACACGGCTTCGACGGCGTTTCTGGCGCGTTTGCAGGGCCATAGACGGCCGCCAGCCCAGCGGCGAGGGCAACCAGCCCGGTGAGCGTCGGAAAGGCGCGCTAGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAGGGCAGGATAGGTGAAGTAGGCCCACCCGCGAGCGGGTGTTCCTTCTTCACTGTCCCTTATTCGCACCTGGCGGTGCTCAACGGGAATCCTGCTCTGCGAGGCTGGCCGGTCACTCTAGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAAGCTTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGCAGTC

/opt/miniconda3/envs/radial_faces_env/lib/python3.8/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (92958925 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Saved: ring_25_linear.png
Ring 26: bases 8822-9484 (662 bases)


/opt/miniconda3/envs/radial_faces_env/lib/python3.8/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (96455324 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Saved: ring_26_linear.png
Ring 27: bases 9484-10169 (685 bases)


...

Saved: ring_27_linear.png
Ring 28: bases 10169-10878 (709 bases)


/opt/miniconda3/envs/radial_faces_env/lib/python3.8/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (103374136 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Saved: ring_28_linear.png
Ring 29: bases 10878-11611 (733 bases)


/opt/miniconda3/envs/radial_faces_env/lib/python3.8/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (106859325 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Saved: ring_29_linear.png
Saved final result: 9_inch_7SQ3_concentric_rings_finalhopefully.png
Success! Created concentric DNA rings.


In [90]:
result = create_dna_concentric_rings(
    sequence=eightE2Q_full_plasmid.upper() + eightE2Q_full_plasmid.upper() + eightE2Q_full_plasmid.upper()+ eightE2Q_full_plasmid.upper(),
    innermost_radius=int(1000 * 1.5),
    ring_height=460,
    margin=10,
    output_file="medium_8E2Q_concentric_rings_finalhopefully.png",
    save_intermediates=True  
)


if result:
    print("Success! Created concentric DNA rings.")
else:
    print("Failed to create rings.")

Processing DNA sequence: GTTGCGGCCACCGGCTGGCTCGCTTCGCTCGGCCCGTGGACAACCCTGCTGGACAAGCTGATGGACAGGCTGCGCCTGCCCACGAGCTTGACCACAGGGATTGCCCACCGGCTACCCAGCCTTCGACCACATACCCACCGGCTCCAACTGCGCGGCCTGCGGCCTTGCCCCATCAATTTTTTTAATTTTCTCTGGGGAAAAGCCTCCGGCCTGCGGCCTGCGCGCTTCGCTTGCCGGTTGGACACCAAGTGGAAGGCGGGTCAAGGCTCGCGCAGCGACCGCGCAGCGGCTTGGCCTTGACGCGCCTGGAACGACCCAAGCCTATGCGAGTGGGGGCAGTCGAAGGCGAAGCCCGCCCGCCTGCCCCCCGAGCCTCACGGCGGCGAGTGCGGGGGTTCCAAGGGGGCAGCGCCACCTTGGGCAAGGCCGAAGGCCGCGCAGTCGATCAACAAGCCCCGGAGGGGCCACTTTTTGCCGGAGGGGGAGCCGCGCCGAAGGCGTGGGGGAACCCCGCAGGGGTGCCCTTCTTTGGGCACCAAAGAACTAGATATAGGGCGAAATGCGAAAGACTTAAAAATCAACAACTTAAAAAAGGGGGGTACGCAACAGCTCATTGCGGCACCCCCCGCAATAGCTCATTGCGTAGGTTAAAGAAAATCTGTAATTGACTGCCACTTTTACGCAACGCATAATTGTTGTCGCGCTGCCGAAAAGTTGCAGCTGATTGCGCATGGTGCCGCAACCGTGCGGCACCCTACCGCATGGAGATAAGCGATGATCGATCGATCGATCGATTCGATCGATCGATCGATCGCGAGAGGGCAGGATAGGTGAAGTAGGCCCACCCGCGAGCGGGTGTTCCTTCTTCACTGTCCCTTATTCGCACCTGGCGGTGCTCAACGGGAATCCTGCTCTGCGAGGCTGGCCGGGATCGATCGATCGATCGATTCGATCGATCGATCGATCGCGAGATTG

/opt/miniconda3/envs/radial_faces_env/lib/python3.8/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (92332286 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Saved: ring_24_linear.png
Ring 25: bases 8640-9297 (657 bases)


/opt/miniconda3/envs/radial_faces_env/lib/python3.8/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (95795055 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Saved: ring_25_linear.png
Ring 26: bases 9297-9978 (681 bases)


...

Saved: ring_26_linear.png
Ring 27: bases 9978-10682 (704 bases)


/opt/miniconda3/envs/radial_faces_env/lib/python3.8/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (102552443 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Saved: ring_27_linear.png
Ring 28: bases 10682-11410 (728 bases)


/opt/miniconda3/envs/radial_faces_env/lib/python3.8/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (106075746 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Saved: ring_28_linear.png
Saved final result: medium_8E2Q_concentric_rings_finalhopefully.png
Success! Created concentric DNA rings.
